In [1]:
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import scipy.optimize 
from datetime import datetime,time,timedelta
from dateutil import relativedelta
from strategy2 import strategy2


In [69]:
def optimize(stock_return, weights,tickers,start,end,calc_window,rf_rate,short=False,
             price_type='Close',clean='True'):
          
        '''
        weights; *type = list, initial guess weights
        
        start; input as str, data start date format 'YYYY-MM-DD' 
        end; input as str, data end date format 'YYYY-MM-DD'
        
        calc_window; input as str, calculation window for returns, 
        
        rf_rate; desired risk-free rate to benchmark against
        
        price_type; *input as str, default = 'Close', options = 'Adj Close', 
        'Close', 'High','Low','Open', 'Volume'
        
        short = bool, default = 'False', if true weight bounds -1 to 1
        
        clean; bool, default = 'False', if true, the function will drop NA rows
        '''
        start = datetime.strptime(start,'%Y-%m-%d')
        end = datetime.strptime(end,'%Y-%m-%d')
        
        prices = pdr.get_data_yahoo(tickers, start, end)[price_type]
        
        op = prices.pct_change(periods=int(calc_window))
        op.iloc[-1,:]=stock_return
        if clean == False:
            op
            
        else:
            op = op.dropna()
        
        vcv_matrix = np.cov(op.T)
        
        avg_return = op.mean()
        
        def sharpe(weights,vcv_matrix,avg_return):        
            
            var = np.dot(np.dot(weights,vcv_matrix),weights)
            
            ret =  np.dot(weights,avg_return)
            
            s_ratio = -(ret-rf_rate)/np.sqrt(var)
            
            return s_ratio
           
        
        sharpe_cons = ({'type':'eq','fun': lambda x: sum(x)-1})
        
        
        if short == False:
            sharpe_bnds = ((0,1),)*len(tickers)
        
        if short == True:
            sharpe_bnds = ((-1,1),)*len(tickers)
        
        optimal = scipy.optimize.minimize(sharpe,weights,bounds = sharpe_bnds,
                                 args = (vcv_matrix,avg_return),
                                 constraints = sharpe_cons)
        
        return optimal

In [2]:
def stock_list_return(date):
    factors = pd.read_csv("../../Factors/merge_Factors.csv")
    factor_returns_pred = pd.read_csv("../../Predict/factors_returns_pred.csv")
    factor_returns_before = pd.read_csv("../../Analysis/factors_returns_multi.csv")
    factor_returns_before = factor_returns_before.loc[factor_returns_before["Date"]<"2022-08-01"]
    factor_returns = factor_returns_before.append(factor_returns_pred)
    
    selected_stocks = strategy2(date)
    factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
    factor_returns = factor_returns.loc[factor_returns["Date"]==date]
    factor_returns = factor_returns.iloc[:,1:]
    stock_list = factors["Index"].tolist()
    factors = factors.iloc[:, 3:]
    factors = np.array(factors)
    factor_returns = np.array(factor_returns)

    stock_return = np.dot(factors, factor_returns.T).reshape(-1)
    stock_list = np.array(stock_list)
    return stock_return, stock_list

In [71]:
def get_weight(date):  

    # factors = pd.read_csv("../../Factors/merge_Factors.csv")
    # factor_returns = pd.read_csv("../../Predict/factors_returns_pred.csv")

    # selected_stocks = strategy2(date)
    # factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
    # factor_returns = factor_returns.loc[factor_returns["Date"]==date]
    # factor_returns = factor_returns.iloc[:,1:]
    # stock_list = factors["Index"].tolist()
    # factors = factors.iloc[:, 3:]
    # factors = np.array(factors)
    # factor_returns = np.array(factor_returns)

    # stock_return = np.dot(factors, factor_returns.T).reshape(-1)
    # stock_list = np.array(stock_list)
    stock_return, stock_list = stock_list_return(date)

    weights=[0.1]*10
    tickers=stock_list
    #end为要预测仓位的那天
    end=date
    #start为end向前一个月
    start=(pd.to_datetime(date)-relativedelta.relativedelta(months=1)).strftime("%Y-%m-%d")
    #不变，始终为1
    calc_window='1'


    # 确定无风险收益率
    rf=pd.read_csv('daily-treasury-rates.csv',index_col=0)

    #rf=rf.iloc[:,5]

    rf.reset_index(inplace=True)

    rf['Date']=[datetime.strptime(i,"%m/%d/%Y") for i in list(rf['Date'].values)]

    start_time=datetime.strptime(start,"%Y-%m-%d")
    end_time=datetime.strptime(end,"%Y-%m-%d")

    rf=rf.set_index("Date")


    for i in range(len(rf.index)):
        if list(rf.index)[i]-end_time<=timedelta(0):
            rfend_time=list(rf.index)[i]
            break
    for j in range(len(rf.index)):
        if list(rf.index)[j]-start_time<=timedelta(0):
            rfstart_time=list(rf.index)[j]
            break

    rf_rate=rf.loc[rfend_time:rfstart_time,'1 Yr'].mean()*0.01/360

    w=optimize(stock_return, weights,tickers,start,end,calc_window,rf_rate,short=True,price_type='Adj Close',clean='True')
    weight=list(w.x)
    return weight

In [77]:
def get_daily_return(date, stock_list):
    factors = pd.read_csv("../../Factors/merge_Factors.csv")

    # factor_returns_pred = pd.read_csv("../../Predict/factors_returns_pred.csv")
    # factor_returns_before = pd.read_csv("../../Analysis/factors_returns_multi.csv")
    # factor_returns_before = factor_returns_before.loc[factor_returns_before["Date"]<"2022-08-01"]
    # factor_returns = factor_returns_before.append(factor_returns_pred)

    # selected_stocks = stock_list
    # factors = factors.loc[(factors["Date"]==date) & (factors["Index"].isin(selected_stocks))]
    # factor_returns = factor_returns.loc[factor_returns["Date"]==date]
    # factor_returns = factor_returns.iloc[:,1:]
    # stock_list = factors["Index"].tolist()
    # factors = factors.iloc[:, 3:]
    # factors = np.array(factors)
    # factor_returns = np.array(factor_returns)

    # stock_return = np.dot(factors, factor_returns.T).reshape(-1)

    # stock_return, stock_list = stock_list_return(date)
    factors = factors.iloc[:, :3]
    factors = factors.loc[factors["Date"]==date]
    factors = factors[factors["Index"].isin(stock_list)]
    true_return = np.array(factors["Return"])

    weight = get_weight(date)
    weight = np.array(weight)
    daily_return = sum(true_return * weight)
    return daily_return

In [73]:
def get_monthly_return(date):
    factors = pd.read_csv("../../Factors/merge_Factors.csv")
    stock_return, stock_list = stock_list_return(date)

    date = pd.to_datetime(date)
    next_month_date = pd.to_datetime(date)+relativedelta.relativedelta(months=1)
    dates = factors["Date"].unique()
    dates = pd.to_datetime(dates)
    dates = dates[dates>=date]
    dates = dates[dates<next_month_date]
    dates = dates.strftime("%Y-%m-%d").tolist()

    monthly_return = []
    for daily in dates:
        daily_return = get_daily_return(daily, stock_list)
        monthly_return.append(daily_return)

    
    return monthly_return

In [116]:
month_10=get_monthly_return("2022-10-03")

In [85]:
month_8=get_monthly_return("2022-08-01")

In [88]:
month_9=get_monthly_return("2022-09-01")

In [94]:
def annal_vala(r):
    return (r.var()**0.5)*(252**0.5)

In [95]:
def month_to_year(t,end_net):
    k=12/t
    r=end_net-1
    year_r=(1+r)**k-1
    return year_r

In [102]:
def get_end_net(r):
    return (1+r).prod()

In [119]:
def get_sharpe_ratio(r):
    r=np.array(r)
    vala = annal_vala(r)
    end_net = get_end_net(r)
    annual_return = month_to_year(1,end_net)
    sharpe_ratio = annual_return/vala
    return sharpe_ratio


In [126]:
month_9 = np.array(month_9)
month_to_year(1,get_end_net(month_9))

0.5636570568454364

In [132]:
month_8_9 = np.concatenate((month_8,month_9))
# month_to_year(2,get_end_net(month_8_9))
get_sharpe_ratio(month_8_9)
# month_8_9


-0.6454471704102179

In [141]:
month_to_year(1,get_end_net(month_7))

5.988066064991004

In [25]:
stock_list=stock_list_return("2022-10-03")[1]
data = pd.read_csv("../../Data/Raw_Stock_Data.csv")
# data = data.loc[data["Date"]>="2022-01-01"]
data = data.loc[data["Index"].isin(stock_list)]
data["Date"] = pd.to_datetime(data["Date"])
# pd.to_datetime(list(date)[0])
data=data[data["Date"]>="2022-01-01"]
data.to_csv("../../Data/Stock_Data.csv",index=False)


In [144]:
stock_list_return("2022-10-03")

(array([0.0045443 , 0.00479362, 0.00728543, 0.00620203, 0.0119694 ,
        0.00292618, 0.00379059, 0.00727838, 0.01113666, 0.01341925]),
 array(['AFBI', 'BSBK', 'CFB', 'CLBK', 'FBNC', 'FFBW', 'FSEA', 'LSBK',
        'MGYR', 'RBKB'], dtype='<U4'))

In [114]:
month_8=np.array(month_8)
month_9=np.array(month_9)
month_10=np.array(month_10)
end_net_8=get_end_net(month_8)
end_net_9=get_end_net(month_9)
end_net_10=get_end_net(month_10[1:])
end_net_10

1.0177496943199449

In [96]:
annal_vala_8=annal_vala(month_8)
annal_vala_9=annal_vala(np.array(month_9))
annal_vala_10=annal_vala(np.array(month_10))


In [113]:
month_10 = month_10[1:]

In [93]:
month_7=[-0.006635350012707567, -0.026298679660253853, 0.02150929351685962, 0.0550507044268199, -0.0005857694477037195,
 0.007334904652437831,
 -0.0027157153770715867,
 -0.0007002721679409494,
 -0.0108382936960135,
 -0.020304300368357238,
 0.00413035201490675,
 0.036357926965512995,
 -0.00623515090206255,
 -0.013789459453768983,
 -0.0210883583279326,
 0.07091067275902574,
 0.0010508139722739144,
 0.023120341033698328,
 0.027581781493120704,
 0.031029754967263873]


In [91]:
len(month_7)

20

In [86]:
len(month_10)

21

In [117]:
month_10

[-0.027237308941689912,
 0.009878868697117462,
 0.010058773507794131,
 0.006454300284833021,
 0.003523193136971999,
 0.006430807396999335,
 0.018873175678441975,
 -0.004861983719556998,
 -0.01666341727128669,
 -0.011209998397873762,
 0.0011849934316640995,
 -0.001971901604343789,
 0.003256704396755181,
 0.006442268222968917,
 -0.02696881755366026,
 0.007564664686684184,
 -0.0014825007416684273,
 0.005575715700259659,
 0.006788722908656116,
 0.007397132151476922,
 0.008045625240174721]

In [104]:
month_8

[-0.008109281099863856,
 -0.004852236542205097,
 0.0037223885168672383,
 0.015895062684167807,
 -0.006972058683159286,
 -0.004331615871663351,
 0.0063403347420709945,
 0.012219770112935437,
 0.018413226439581747,
 0.020939780522183957,
 -0.014523049477755335,
 -0.006091626340628034,
 0.010268694204830913,
 0.010576630760128986,
 -0.023496995018869295,
 -0.005114979521992628,
 -0.016778908475407386,
 -0.0068444247607942895,
 0.025768405219062287,
 -0.018529203609827974,
 0.008703702193835452,
 -0.014926616483171527,
 0.03618064645729547]